In [1]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch

device = 'cpu' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' or device == 'mps' else False)


/opt/homebrew/anaconda3/envs/bark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/bark/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [2]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
from hubert.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed(download_url = 'https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960.pt', file_name = 'hubert.pt')
hubert_manager.make_sure_tokenizer_installed(model= 'quantifier_hubert_base_ls960_14.pth', repo = 'GitMylo/bark-voice-cloning', local_file = 'tokenizer.pth')
hubert_manager.make_sure_tokenizer_installed(model = 'german-HuBERT-quantizer_14_epoch.pth', repo = 'CountFloyd/bark-voice-cloning-german-HuBERT-quantizer', local_file = 'tokenizer_german.pth')


'models/hubert/tokenizer_german.pth'

In [3]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer 
# Load HuBERT for semantic tokens
from hubert.pre_kmeans_hubert import CustomHubert
from hubert.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('models/hubert/tokenizer_german.pth',map_location=device).to(device)  # Automatically uses the right layers

2025-02-15 15:38:01 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [4]:
# Load and pre-process the audio waveform
audio_filepath = 'source_voices/christian_brueckner_versuchung_von_syrakus.wav' # the audio you want to clone
wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [5]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [6]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [7]:
# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [10]:
import numpy as np
voice_name = 'de_brueckner_0' # whatever you want the name of the voice to be
output_path = 'prepared_voices/' + voice_name + '.npz'
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)